#### Q1. Install MLflow
To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?



In [1]:
! mlflow --version

mlflow, version 1.26.0


#### Q2. Download and preprocess the data
We'll use the same dataset used in the videos of the 1st and 2nd module: "Green Taxi Trip Records".


1. Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

2. Use the script *preprocess_data.py* located in the folder homework to preprocess the data.

        The script will 
        - load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
        - fit a DictVectorizer on the training set (January 2021 data), 
        - save the preprocessed datasets and the model to disk


Your task is to download the datasets and then execute this command:

```bash
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

~~Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.~~

How many files were saved to OUTPUT_FOLDER?

- [] 1
- [] 3
- [x] 4
- [] 7

In [ ]:
RAW_DATA_PATH ="./artifacts/data/raw"
PROCESSED_DATA_PATH ="./artifacts/data/processed"

In [2]:
# Step 1: Install Data
# Prepare Environment
!mkdir artifacts
!mkdir artifacts/data
!mkdir artifacts/data/raw
!mkdir artifacts/data/processed


# Download 2021 January, February and March data
!python download_file.py  --raw_data_path ./artifacts/data/raw --file_identifier 2021-01
!python download_file.py  --raw_data_path ./artifacts/data/raw --file_identifier 2021-02
!python download_file.py  --raw_data_path ./artifacts/data/raw --file_identifier 2021-03

mkdir: artifacts: File exists
mkdir: artifacts/data: File exists
mkdir: artifacts/data/raw: File exists
mkdir: artifacts/data/processed: File exists
https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2021-01.parquet
https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2021-02.parquet
https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2021-03.parquet


In [3]:
# Step 2: Preprocess Data
!python preprocess_data.py --raw_data_path ./artifacts/data/raw --dest_path ./artifacts/data/processed

#### Q3. We will train a random forest regressor on the taxi dataset.

We have prepared the training script *train.py* for this exercise, which can be also found in the folder homework. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.


- Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

- Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

- [] 19
- [x] 17
- [] 10
- [] 20

In [5]:
! python train.py

2022/05/28 00:11:23 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/28 00:11:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/iremertuerk/workspace-personal/mlops-zoomcamp/.venv/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


#### Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to
- launch the tracking server on your local machine
- select a SQLite db for the backend store and a folder called artifacts for the artifacts store

PS: You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

- [x] default-artifact-root
- [x] serve-artifacts
- [] artifacts-only
- [x] artifacts-destination

In [ ]:
! mlflow ui --backend-store-uri sqlite:///mlflow.db --artifacts-destination ./artifacts/mlartifacts --serve-artifacts

READ: 
- [mlflow-with-remote-tracking-server-backend-and-artifact-stores](https://www.mlflow.org/docs/latest/tracking.html#scenario-4-mlflow-with-remote-tracking-server-backend-and-artifact-stores)

#### Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using hyperopt. We have prepared the script `hpo.py` for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization Tip:(you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Hints: Don't use autologging for this exercise.The idea is to just log the information that you need to answer the question below, including:
- the list of hyperparameters that are passed to the objective function during the optimization.
- the RMSE obtained on the validation set (February 2021 data).


What's the best validation RMSE that you got?

- [] 6.128
- [x] 6.628
- [] 7.128
- [] 7.628


In [7]:
! python hpo.py

100%|█████████| 50/50 [02:47<00:00,  3.35s/trial, best loss: 6.6284257482044735]


#### Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the `MlflowClient` to get the model with the lowest RMSE. 
Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `runs:/<RUN_ID>/model`, and the `name of the model` (make sure to choose a good one!).

What is the test RMSE of the best model?

- [] 6.1
- [x] 6.55
- [] 7.93
- [] 15.1

In [8]:
! python register_model.py

2022/05/28 00:19:08 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/28 00:19:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/iremertuerk/workspace-personal/mlops-zoomcamp/.venv/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
<Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='random-forest-best-models', tags={}>
Best Run <RunInfo: artifact_uri='./mlruns/3/1d60d49a732f4db9ab3fe297903d35a2/artifacts', end_time=1653689983901, experiment_id='3', lifecycle_stage='active', run_id='1d60d49a732f4db9ab3fe297903d35a2', run_uuid='1d60d49a732f4db9ab3fe297903d35a2', start_time=1653689975431, status='FINISHED', user_id='iremertuerk'>
Registered model 'RandomForestRegressionMode